In [4]:
# ! pip install torchfm

In [5]:
import torch
import tqdm
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader
import os
import pandas as pd

from torchfm.model.afi import AutomaticFeatureInteractionModel
from torchfm.model.afm import AttentionalFactorizationMachineModel
from torchfm.model.dcn import DeepCrossNetworkModel
from torchfm.model.dfm import DeepFactorizationMachineModel
from torchfm.model.ffm import FieldAwareFactorizationMachineModel
from torchfm.model.fm import FactorizationMachineModel
from torchfm.model.fnfm import FieldAwareNeuralFactorizationMachineModel
from torchfm.model.fnn import FactorizationSupportedNeuralNetworkModel
from torchfm.model.lr import LogisticRegressionModel
from torchfm.model.ncf import NeuralCollaborativeFiltering
from torchfm.model.nfm import NeuralFactorizationMachineModel
from torchfm.model.pnn import ProductNeuralNetworkModel
from torchfm.model.wd import WideAndDeepModel
from torchfm.model.xdfm import ExtremeDeepFactorizationMachineModel
from torchfm.model.afn import AdaptiveFactorizationNetwork

In [6]:
new_df = pd.read_csv(os.path.join("..","..","data","input_data.csv"))
new_df.head()

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user.id,place.id,place.name,category,place.category,user.rating,place.rating,createdDate,is_fch,photonum,review.count,language,lat,lng,land.addr,review,is_local
0,4TXey,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,4.5,4.590717,2020-05-21 20:42:00,1,0.0,158,ko,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1
1,awWC,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2019-06-28 01:11:03,1,0.0,158,ko,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",none,1
2,1Unzg,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2020-03-12 00:22:00,1,0.0,158,ko,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1
3,LRVl,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2020-02-16 17:06:00,1,0.0,158,ko,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1
4,ESN1,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2019-03-11 00:08:48,1,0.0,158,ko,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",2019.03.06\nHH,1


In [14]:
new_df = pd.read_csv(os.path.join("..","..","data","TA_User_Reviws_Korea_all_v2_new_df.csv"))
new_df.head()

,locationId,rated_count,average_photonum,average_rating,userID,users_mean_rating,user_reviewcount,rating,location_name,placeType,createdDate,islocal,photonum
0,299152,13,2.846154,3.692308,4842,3.677083,96,5,Best Western Premier Seoul Garden Hotel,ACCOMMODATION,20171009,1,12
1,299152,13,2.846154,3.692308,9212,3.791667,240,3,Best Western Premier Seoul Garden Hotel,ACCOMMODATION,20180306,1,0
2,299152,13,2.846154,3.692308,14262,4.500000,2,4,Best Western Premier Seoul Garden Hotel,ACCOMMODATION,20121220,0,1
3,299152,13,2.846154,3.692308,20235,3.000000,4,1,Best Western Premier Seoul Garden Hotel,ACCOMMODATION,20161127,0,4
4,299152,13,2.846154,3.692308,21432,4.200000,5,4,Best Western Premier Seoul Garden Hotel,ACCOMMODATION,20120416,0,0


In [17]:
new_df.columns

Index(['locationId', 'rated_count', 'average_photonum', 'average_rating',
       'userID', 'users_mean_rating', 'user_reviewcount', 'rating',
       'location_name', 'placeType', 'createdDate', 'islocal', 'photonum'],
      dtype='object')

In [ ]:
x = ['locationId', 'rated_count', 'average_photonum', 'average_rating',
       'userID', 'users_mean_rating', 'user_reviewcount', 
        'createdDate', 'photonum']
y = 'rating'

groups = []
labels = []

for i in range(new_df.shape[0]):
    groupsa.append(new_df.iloc[i,x])
    labels.append(new_df.iloc[i,y])

In [15]:
for x in new_df.iloc[:3,:]:
    print(x)

locationId
rated_count
average_photonum
average_rating
userID
users_mean_rating
user_reviewcount
rating
location_name
placeType
createdDate
islocal
photonum


In [7]:
def get_dataset(name):
    if name == 'TripAdvisor':
        new_df = pd.read_csv(os.path.join("..","..","data","TA_User_Reviws_Korea_all_v2_new_df.csv"))
        new_df = new_df.drop(['location_name'], axis=1)
        return new_df
    elif name == "FinalData":
        new_df = pd.read_csv(os.path.join("..","..","data","input_data.csv"))
#         new_df = new_df.drop(['location_name'], axis=1)
        return new_df
    else:
        raise ValueError('unknown dataset name: ' + name)

In [8]:
def get_model(name, dataset):
    """
    Hyperparameters are empirically determined, not opitmized.
    """
    field_dims = dataset.field_dims
    if name == 'lr':
        return LogisticRegressionModel(field_dims)
    elif name == 'fm':
        return FactorizationMachineModel(field_dims, embed_dim=16)
    elif name == 'ffm':
        return FieldAwareFactorizationMachineModel(field_dims, embed_dim=4)
    elif name == 'fnn':
        return FactorizationSupportedNeuralNetworkModel(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'wd':
        return WideAndDeepModel(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'ipnn':
        return ProductNeuralNetworkModel(field_dims, embed_dim=16, mlp_dims=(16,), method='inner', dropout=0.2)
    elif name == 'opnn':
        return ProductNeuralNetworkModel(field_dims, embed_dim=16, mlp_dims=(16,), method='outer', dropout=0.2)
    elif name == 'dcn':
        return DeepCrossNetworkModel(field_dims, embed_dim=16, num_layers=3, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'nfm':
        return NeuralFactorizationMachineModel(field_dims, embed_dim=64, mlp_dims=(64,), dropouts=(0.2, 0.2))
    elif name == 'ncf':
        # only supports MovieLens dataset because for other datasets user/item colums are indistinguishable
        assert isinstance(dataset, MovieLens20MDataset) or isinstance(dataset, MovieLens1MDataset)
        return NeuralCollaborativeFiltering(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.2,
                                            user_field_idx=dataset.user_field_idx,
                                            item_field_idx=dataset.item_field_idx)
    elif name == 'fnfm':
        return FieldAwareNeuralFactorizationMachineModel(field_dims, embed_dim=4, mlp_dims=(64,), dropouts=(0.2, 0.2))
    elif name == 'dfm':
        return DeepFactorizationMachineModel(field_dims, embed_dim=16, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'xdfm':
        return ExtremeDeepFactorizationMachineModel(
            field_dims, embed_dim=16, cross_layer_sizes=(16, 16), split_half=False, mlp_dims=(16, 16), dropout=0.2)
    elif name == 'afm':
        return AttentionalFactorizationMachineModel(field_dims, embed_dim=16, attn_size=16, dropouts=(0.2, 0.2))
    elif name == 'afi':
        return AutomaticFeatureInteractionModel(
             field_dims, embed_dim=16, atten_embed_dim=64, num_heads=2, num_layers=3, mlp_dims=(400, 400), dropouts=(0, 0, 0))
    elif name == 'afn':
        print("Model:AFN")
        return AdaptiveFactorizationNetwork(
            field_dims, embed_dim=16, LNN_dim=1500, mlp_dims=(400,400,400), dropouts=(0, 0, 0))
    else:
        raise ValueError('unknown model name: ' + name)

In [9]:
def train(model, optimizer, data_loader, criterion, device, log_interval=1000):
    model.train()
    total_loss = 0
    for i, (fields, target) in enumerate(tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0)):
        fields, target = fields.to(device), target.to(device)
        y = model(fields)
        loss = criterion(y, target.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (i + 1) % log_interval == 0:
            print('    - loss:', total_loss / log_interval)
            total_loss = 0

In [10]:
device = 'cuda:0'
dataset_name= 'TripAdvisor'
model_name= 'afm',
epoch= 2400,
learning_rate= 1e-4,
batch_size= 2400,
weight_decay= 1e-6

In [11]:
dataset = get_dataset(dataset_name)
train_length = int(len(dataset) * 0.9)
valid_length = int(len(dataset) * 0.1)

print(len(dataset), train_length, valid_length)

137490 123741 13749


In [12]:
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, (train_length, valid_length))
train_data_loader = DataLoader(train_dataset, batch_size=batch_size[0], num_workers=8)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size[0], num_workers=8)

In [13]:
model = get_model(model_name, dataset).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.RMSprop(params=model.parameters(), lr=learning_rate, weight_decay=weight_decay)

AttributeError: 'DataFrame' object has no attribute 'field_dims'

In [ ]:
for epoch_i in range(epoch):
    train(model, optimizer, train_data_loader, criterion, device)
    print('epoch:', epoch_i, 'validation: auc:', auc)

In [54]:
# def main(dataset_name, model_name, epoch, learning_rate, batch_size, weight_decay, device):
#     device = torch.device(device)
#     dataset = get_dataset(dataset_name)
#     train_length = int(len(dataset) * 0.9)
#     valid_length = int(len(dataset) * 0.1)
# #     test_length = len(dataset) - train_length - valid_length
#     train_dataset, valid_dataset = torch.utils.data.random_split(
#         dataset, (train_length, valid_length))
#     train_data_loader = DataLoader(train_dataset, batch_size=batch_size[0], num_workers=8)
#     valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size[0], num_workers=8)
# #     test_data_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=8)
#     model = get_model(model_name, dataset).to(device)
#     criterion = torch.nn.MSELoss()
#     optimizer = torch.optim.RMSprop(params=model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#     for epoch_i in range(epoch):
#         train(model, optimizer, train_data_loader, criterion, device)
# #         auc = test(model, valid_data_loader, device)
#         print('epoch:', epoch_i, 'validation: auc:', auc)
# #     auc = test(model, test_data_loader, device)
# #     print('test auc:', auc)
# #     torch.save(model, f'{save_dir}/{model_name}.pt')

# main(dataset_name, model_name, epoch,
#          learning_rate, batch_size, weight_decay, device) 